<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Asi_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, place the data files somewhere in your drive so that you can access the files from this notebook. The files are available to download from the Moodle assignment activity*

The relevant files are:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [0]:
import json
from pathlib import Path
from glob import glob
import pandas as pd
import os

os.chdir('/content/drive/My Drive/idc/nlp/ex1/')

def preprocess():
  vocabulary = set()
  pathlist = glob('*.csv')
  for data_file_path in pathlist:
      data_file = pd.read_csv(data_file_path)
      for data in data_file['tweet_text'].values:
          vocabulary.update(list(data))

  vocabulary.add('<s>')
  vocabulary.add('</s>')

  # vocabulary.discard(' ')
  # vocabulary.discard('\t')
  # vocabulary.discard('\r')
  # vocabulary.discard('\n')

  return sorted(list(vocabulary))

  

In [3]:
%%time
vocabulary = preprocess()
print(len(vocabulary))
print(vocabulary)

1861
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '</s>', '<s>', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x91', '\x92', '\x97', '\x9d', '¡', '£', '¤', '¥', '§', '¨', '©', 'ª', '«', '\xad', '®', '¯', '°', '²', '³', '´', '¶', '·', '¸', 'º', '»', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Å', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ù', 'Ú', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ė', 'Ğ', 'ğ', 'İ', 'ı', 'ń', 'ō', 'Œ', 'œ', 'Ş', 'ş', 'Š', 'Ÿ', 'ƒ', 'ʔ', 'ʕ', '

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [0]:
from collections import defaultdict, UserDict

class dict_value(UserDict):
    def __init__(self, value=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.value = value

    def __missing__(self, key):
        return self.value

def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  voc_size = len(vocabulary)
  model = defaultdict(lambda: defaultdict(lambda: 0))
  data_file = pd.read_csv(data_file_path)
  for data in data_file['tweet_text'].values:
      data = ["<s>"] + list(data) + ["</s>"]
      for i in range(len(data) - n + 1):
          word, char = ''.join(data[i:i + n - 1]), data[i + n - 1]
          model[word][char] += 1

  if add_one:
      pmodel = defaultdict(lambda: defaultdict(lambda: 1 / voc_size))
  else:
      pmodel = defaultdict(lambda: defaultdict(lambda: 1e-08))
  for word, counts in model.items():
      if add_one:
          total_count = sum(counts.values()) + voc_size
          pmodel[word] = dict_value(1/total_count)
          for char in counts:
              pmodel[word][char] = (counts[char] + 1) / total_count
      else:
          total_count = sum(counts.values())
          for char in counts:
              pmodel[word][char] = counts[char] / total_count

  return pmodel



In [5]:
%%time
vocabulary = preprocess()
model = lm(3, vocabulary, 'en.csv', False)
print(model['Ab'])
print(sum(v for v in model['Ab'].values()))
print(sum(model['Ab'][v] for v in vocabulary))

model = lm(3, vocabulary, 'en.csv', True)
print(model['Ab'])
print(sum(v for v in model['Ab'].values()))
print(sum(model['Ab'][v] for v in vocabulary))


defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7f2dab1d3bf8>, {'o': 0.2571428571428571, 's': 0.1, 'e': 0.05714285714285714, 'U': 0.04285714285714286, 'u': 0.08571428571428572, 'D': 0.014285714285714285, 'd': 0.1, 'a': 0.014285714285714285, 'b': 0.1, '7': 0.014285714285714285, 'y': 0.014285714285714285, 'l': 0.02857142857142857, 'j': 0.014285714285714285, 'q': 0.014285714285714285, '2': 0.014285714285714285, 'i': 0.02857142857142857, '</s>': 0.014285714285714285, 'r': 0.05714285714285714, 'n': 0.014285714285714285, 'Z': 0.014285714285714285})
0.9999999999999994
1.0000184099998928
{'o': 0.00983946141895391, 's': 0.004142931123770067, 'e': 0.002589331952356292, 'U': 0.0020714655618850335, 'u': 0.0036250647332988087, 'D': 0.0010357327809425167, 'd': 0.004142931123770067, 'a': 0.0010357327809425167, 'b': 0.004142931123770067, '7': 0.0010357327809425167, 'y': 0.0010357327809425167, 'l': 0.0015535991714137752, 'j': 0.0010357327809425167, 'q': 0.0010357327809425167, '2': 0.0

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [0]:
import math

def eval(n, model, data_file_path):
  # n - the n-gram that you used to build your model (must be the same number)
  # model - the dictionary (model) to use for calculating perplexity
  # data_file - the tweets file that you wish to claculate a perplexity score for

  data_file = pd.read_csv(data_file_path)
  count = 0
  total = 0.0
  for data in data_file['tweet_text'].values:
      data = ['<s>'] + list(data) + ['</s>']
      for i in range(len(data) - n + 1):
          count += 1
          word, char = ''.join(data[i:i + n - 1]), data[i + n - 1]
          total += math.log2(model[word][char])

  ent = -1 / count * total
  per = 2 ** ent

  return per

In [7]:
%%time
model = lm(3, vocabulary, 'en.csv', False)
print(eval(3, model, 'en.csv'))

8.965438001439026
CPU times: user 1.18 s, sys: 8.87 ms, total: 1.19 s
Wall time: 1.2 s


In [8]:
%%time
model = lm(3, vocabulary, 'en.csv', True)
print(eval(3, model, 'en.csv'))

26.318871163182674
CPU times: user 1.47 s, sys: 10 ms, total: 1.48 s
Wall time: 1.48 s


**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [0]:
def match(n, add_one):
  # n - the n-gram to use for creating n-gram models
  # add_one - use add_one smoothing or not

  file_path = '{}.csv'
  vocabulary = preprocess()

  lang = ['en' ,'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
  df = pd.DataFrame(columns=lang, index=lang)
  for l1 in lang:
    l1_model = lm(n, vocabulary, file_path.format(l1), add_one)
    for l2 in lang:
      df.at[l1, l2] = eval(n, l1_model, file_path.format(l2))

  return df


In [10]:
%%time
pd.options.display.max_columns = None
print(match(3, False))

         en       es       fr       in       it       nl       pt       tl
en  8.96544  78.0531  109.272  93.1641  65.1218  87.6969   106.41  76.1182
es  78.8428   8.5912  101.603  134.955  52.6994  143.156  57.7033  101.265
fr  58.3061  66.1363  8.56056  106.203  56.6941  99.7183  86.5802  98.8763
in  57.6622  86.8521  145.839  9.87273  74.3799  89.9082    108.3   59.971
it  73.2088  59.7344  95.8546  144.067  8.60361  144.459  74.6154  93.2924
nl  49.1501   86.677  95.5061  88.6947  76.5365  9.16546  110.144  79.1393
pt  97.6712  55.0793  124.818  178.722  69.2821  185.273  8.05634  117.905
tl  50.5246  74.3102   154.86  64.0306    69.47  112.556  98.4439  8.50025
CPU times: user 41.8 s, sys: 155 ms, total: 41.9 s
Wall time: 42.1 s


In [11]:
%%time
pd.options.display.max_columns = None
print(match(3, True))

         en       es       fr       in       it       nl       pt       tl
en  26.3189  57.2445  57.8933  77.0167    59.52  60.3326  67.5199  70.3138
es  71.0303  24.2023  59.6484  89.7969  48.3866  82.4741  46.4629  88.0705
fr  57.9458  48.3237   24.232  85.3929  53.6109  68.9781  59.6998  88.1311
in  62.2854  66.4444  77.0803  30.4108  67.3064  71.3141  75.6373  57.3578
it  69.0617  46.6899  60.6116  92.5048  24.9038   86.205  53.3553   80.954
nl  53.5112   68.309   64.354  78.1774   71.831  27.1713   78.419  77.2418
pt  80.3243  45.3119  68.1943  103.263  57.5052  97.3543  25.4544  93.9566
tl  54.0615  63.0378  76.8159  58.7234   63.866  76.7101  72.8183  28.5867
CPU times: user 58.6 s, sys: 138 ms, total: 58.7 s
Wall time: 58.9 s


**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [12]:
%%time
pd.options.display.max_columns = None
vocabulary = preprocess()
for n in range(0, 4):
  print("\n\nn: {}. add_one: False".format(n + 1))
  print(match(n + 1, False))
  print("\n\nn: {}. add_one: True".format(n + 1))
  print(match(n + 1, True))



n: 1. add_one: False
         en       es       fr       in       it       nl       pt       tl
en  38.5819  41.7284  43.9312  41.7317  41.2459  39.7133  45.2394  45.0282
es  42.1904  36.2134  41.5859  43.8699  40.4491  41.5543  41.0399  47.5528
fr   41.802   40.208  37.5004  44.6866  39.9451  40.9627  42.4084   49.471
in  42.6906  45.4138  48.1969  37.3912  43.8748  41.9243  48.3191  42.9053
it  41.6723  39.0687  40.9179  43.6638  37.5525  41.0965  42.6782  46.7371
nl  40.9808  41.1384  42.1104   41.815  41.1531  37.4997  43.9915  46.7566
pt  42.7496  37.6343  41.0913  43.2408  41.0032  41.7144  37.0763   47.514
tl  42.4023  43.1284   48.603  39.3321  43.3317  42.8451   45.963  40.7645


n: 1. add_one: True
         en       es       fr       in       it       nl       pt       tl
en  38.6302  40.8316  42.7346  41.5393  40.5401  39.6455   43.361  44.8704
es  42.0466  36.2646  40.8816  43.6463  39.9983  41.4236  39.7485  47.3664
fr   41.632  39.7507   37.546  44.4726  39.7949  40.872

# **Good luck!**